In [ ]:
# Code to create a large amount of CIFAR successful adversarial examples 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import math
import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 
import prebuilt_attacks as pa

In [ ]:
# LOAD THE PRETRAINED (DEFENDED) MODEL (NOTE: THIS HAS TO BE RESNET32)

USE_GPU = torch.cuda.is_available()

VALID_EXPERIMENTS = ['fgsm', 'fullSuite', 'linf', 'linfStadv', 'resnetStadv', 'RTDelta', 
                     'singleFlowDelta']

EXPERIMENT_NAME = 'linfStadv'


classifier_net, cifar_normer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, use_gpu=USE_GPU, 
                                                                    return_normalizer=True) 
classifier_net.cuda()

if EXPERIMENT_NAME != 'undefended':
    defended_state_dict_file = checkpoints.params_to_filename(EXPERIMENT_NAME, 'resnet32')[-1]
    print "Loading %s params into resnet32" % defended_state_dict_file 
    classifier_net = checkpoints.load_state_dict_from_filename(defended_state_dict_file, classifier_net)

#print

In [ ]:
# LOAD THE EVALUATION DATASET 
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=128, use_gpu=USE_GPU, 
                                         shuffle=True, no_transform=True)

iter_val = iter(val_loader)
for i in xrange(10):
    next(iter_val)
    
examples, labels = next(iter_val)
examples = examples.cuda()
labels = labels.cuda()

PGD_KWARGS = {'num_iterations': (50, 200), 
              'signed': False, 
              'optimizer': optim.Adam, 
              'optimizer_kwargs': {'lr': 0.001}, 
              'verbose': False, 
              'loss_convergence': 0.999999}

In [ ]:
ATTACK_LIST = ['delta_fgsm', 'delta_pgd', 'stadv', 'rot_trans', 
               'delta_rot_trans', 'delta_stadv',
              # 'delta_stadv_rot_trans'
              ]
OUTPUT = 'attack'
def build_attack_ensemble(model, normalizer, attack_list=None,
                          delta_bound=pa.L_INF_BOUND, 
                          flow_bound=pa.FLOW_LINF, 
                          rot_bound=pa.ROT_LINF,
                          trans_bound=pa.TRANS_LINF):
    attack_list = attack_list or ATTACK_LIST
    attack_ensemble = {} 
    
    if 'delta_pgd' in attack_list:
        attack_ensemble['delta_pgd'] = pa.build_delta_pgd(model, normalizer,
                                                          linf_bound=delta_bound,
                                                          output=OUTPUT)
    if 'stadv' in attack_list: 
        attack_ensemble['stadv'] = pa.build_stadv_pgd(model, normalizer,
                                                      linf_bound=flow_bound,
                                                      output=OUTPUT)
    
    if 'rot_trans' in attack_list:
        attack_ensemble['rot_trans'] = pa.build_rot_trans_pgd(
                                          model, normalizer, 
                                          rot_bound=rot_bound, 
                                          trans_bound=trans_bound,
                                          output=OUTPUT)
        
    if 'delta_rot_trans' in attack_list:
        attack_ensemble['delta_rot_trans'] = pa.build_delta_rot_trans_pgd(
                                              model, normalizer, 
                                              delta_bound=delta_bound, 
                                              trans_bound=trans_bound, 
                                              rot_bound=rot_bound,
                                              output=OUTPUT)

    if 'delta_stadv' in attack_list:
        attack_ensemble['delta_stadv'] = pa.build_delta_stadv_pgd(
                                            model, normalizer, 
                                            delta_bound=delta_bound, 
                                            flow_bound=flow_bound, 
                                            output=OUTPUT)
    
    if 'delta_stadv_rot_trans' in attack_list:
        attack_ensemble['delta_stadv_rot_trans'] =\
            pa.build_delta_stadv_rot_trans_pgd(model, normalizer, 
                                               delta_bound=delta_bound, flow_bound=flow_bound, 
                                               rot_bound=rot_bound, trans_bound=trans_bound, 
                                               output=OUTPUT)
    
    return attack_ensemble
    


In [ ]:
attack_ensemble = build_attack_ensemble(classifier_net, cifar_normer, 
                                        delta_bound=8.0 / 255, 
                                        flow_bound=0.03)

perts = {} 
for atk, atk_obj in attack_ensemble.items():
    perts[atk] = atk_obj.attack(examples, labels, **PGD_KWARGS)
    

In [ ]:
#DISPLAY IMAGES: pick which perturbation style you want to display
PERT_STYLE = 'delta_stadv'
NUM_TO_SHOW = 8
to_display = perts[PERT_STYLE].collect_adversarially_successful(classifier_net, cifar_normer, labels)
adversarials, originals = to_display['adversarial'][:NUM_TO_SHOW], to_display['originals'][:NUM_TO_SHOW]
diffs = torch.clamp((adversarials - originals) * 5 + 0.5, 0, 1)

img_utils.show_images([adversarials, originals, diffs])
#adversarials[:16].shape